In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# Read in the country_vaccinations CSV file as Pandas DataFrames.
vaccinations_raw_data = pd.read_csv('Resources/country_vaccinations.csv', low_memory=False)
vaccinations_raw_data


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7483,Zimbabwe,ZWE,2021-03-16,39550.0,39550.0,NaN,1890.0,541.0,0.27,0.27,NaN,36.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...
7484,Zimbabwe,ZWE,2021-03-17,39607.0,39607.0,NaN,57.0,529.0,0.27,0.27,NaN,36.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...
7485,Zimbabwe,ZWE,2021-03-18,40002.0,40002.0,NaN,395.0,569.0,0.27,0.27,NaN,38.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...
7486,Zimbabwe,ZWE,2021-03-19,41687.0,41687.0,NaN,1685.0,772.0,0.28,0.28,NaN,52.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...


In [3]:
# Drop the NaN data
vaccinations_clean_data=vaccinations_raw_data.dropna()
vaccinations_clean_data

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
62,Albania,ALB,2021-02-18,3049.0,2438.0,611.0,1348.0,254.0,0.11,0.08,0.02,88.0,Pfizer/BioNTech,Ministry of Health,https://shendetesia.gov.al/covid19-ministria-e...
270,Argentina,ARG,2021-01-21,265724.0,249372.0,16352.0,17791.0,11704.0,0.59,0.55,0.04,259.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
271,Argentina,ARG,2021-01-22,279602.0,254456.0,25146.0,13878.0,11263.0,0.62,0.56,0.06,249.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
272,Argentina,ARG,2021-01-23,288064.0,258876.0,29188.0,8462.0,11124.0,0.64,0.57,0.06,246.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
273,Argentina,ARG,2021-01-24,292023.0,260036.0,31987.0,3959.0,10342.0,0.65,0.58,0.07,229.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7452,Wales,OWID_WLS,2021-03-15,1412849.0,1139866.0,272983.0,25663.0,30490.0,44.81,36.15,8.66,9671.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
7453,Wales,OWID_WLS,2021-03-16,1446436.0,1157251.0,289185.0,33587.0,31547.0,45.88,36.70,9.17,10006.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
7454,Wales,OWID_WLS,2021-03-17,1484566.0,1180155.0,304411.0,38130.0,32646.0,47.09,37.43,9.66,10354.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
7455,Wales,OWID_WLS,2021-03-18,1523077.0,1204101.0,318976.0,38511.0,32705.0,48.31,38.19,10.12,10373.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [4]:
# Change column name to match the SQL schema

vaccinations_clean_data=vaccinations_clean_data.rename({'country' : 'country_name'}, axis =1)
vaccinations_clean_data

,country_name,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
62,Albania,ALB,2021-02-18,3049.0,2438.0,611.0,1348.0,254.0,0.11,0.08,0.02,88.0,Pfizer/BioNTech,Ministry of Health,https://shendetesia.gov.al/covid19-ministria-e...
270,Argentina,ARG,2021-01-21,265724.0,249372.0,16352.0,17791.0,11704.0,0.59,0.55,0.04,259.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
271,Argentina,ARG,2021-01-22,279602.0,254456.0,25146.0,13878.0,11263.0,0.62,0.56,0.06,249.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
272,Argentina,ARG,2021-01-23,288064.0,258876.0,29188.0,8462.0,11124.0,0.64,0.57,0.06,246.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
273,Argentina,ARG,2021-01-24,292023.0,260036.0,31987.0,3959.0,10342.0,0.65,0.58,0.07,229.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7452,Wales,OWID_WLS,2021-03-15,1412849.0,1139866.0,272983.0,25663.0,30490.0,44.81,36.15,8.66,9671.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
7453,Wales,OWID_WLS,2021-03-16,1446436.0,1157251.0,289185.0,33587.0,31547.0,45.88,36.70,9.17,10006.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
7454,Wales,OWID_WLS,2021-03-17,1484566.0,1180155.0,304411.0,38130.0,32646.0,47.09,37.43,9.66,10354.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
7455,Wales,OWID_WLS,2021-03-18,1523077.0,1204101.0,318976.0,38511.0,32705.0,48.31,38.19,10.12,10373.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [5]:
# DataFrame for SQL table country_master
vaccination_table = vaccinations_clean_data[['country_name','iso_code','date','total_vaccinations','people_vaccinated','people_fully_vaccinated','daily_vaccinations_raw','daily_vaccinations','total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred']]
vaccination_table

,country_name,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred
62,Albania,ALB,2021-02-18,3049.0,2438.0,611.0,1348.0,254.0,0.11,0.08,0.02
270,Argentina,ARG,2021-01-21,265724.0,249372.0,16352.0,17791.0,11704.0,0.59,0.55,0.04
271,Argentina,ARG,2021-01-22,279602.0,254456.0,25146.0,13878.0,11263.0,0.62,0.56,0.06
272,Argentina,ARG,2021-01-23,288064.0,258876.0,29188.0,8462.0,11124.0,0.64,0.57,0.06
273,Argentina,ARG,2021-01-24,292023.0,260036.0,31987.0,3959.0,10342.0,0.65,0.58,0.07
...,...,...,...,...,...,...,...,...,...,...,...
7452,Wales,OWID_WLS,2021-03-15,1412849.0,1139866.0,272983.0,25663.0,30490.0,44.81,36.15,8.66
7453,Wales,OWID_WLS,2021-03-16,1446436.0,1157251.0,289185.0,33587.0,31547.0,45.88,36.70,9.17
7454,Wales,OWID_WLS,2021-03-17,1484566.0,1180155.0,304411.0,38130.0,32646.0,47.09,37.43,9.66
7455,Wales,OWID_WLS,2021-03-18,1523077.0,1204101.0,318976.0,38511.0,32705.0,48.31,38.19,10.12


In [6]:
# Create the path to your file directory and variable for the file
#file_dir='C:\Users\aksha\Desktop\Class folder\Final project\my work\segment2\Resources\Cleaned_data'

# The final cleaned vaccination data
# vaccinations_clean_data = pd.DataFrame()

# vaccination_cleaned_data = vaccinations_clean_data.to_csv('Resources/Cleaned_data/vaccination_cleaned_data.csv',index =True)
vaccination_table = vaccination_table.to_csv('Resources/Cleaned_data/vaccination_table.csv',index =False)